Import necessary packages

In [1]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
import nest_asyncio
nest_asyncio.apply()


#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "Context_files"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)

#create inputs to the model, telling it what needs to be done. 
#provides system message
system1 = context_gen("system1.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

system3 = context_gen("system3.txt")

system2 = context_gen("system2.txt")

rag_prompt = context_gen("rag_prompt.txt")


#First input to the model, breaks query down and provides geometry definition

class Response(BaseModel):
    response: str= Field(description="YES or NO")
    feature: str= Field(description="geometry")

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [8]:
'''change the human variable to test different inputs'''
human = ("select all vanes")


In [9]:

chat_with_tools = chat.bind_tools([Response])
parser = PydanticToolsParser(tools=[Response])
query_analyzer = ChatPromptTemplate.from_messages([("system", rag_prompt), ("human", human)])

chain = query_analyzer | chat_with_tools | parser
feature_extractor = chain.invoke({}, {"tags": ["loop 001"]})
print(feature_extractor)

feature = []
for item in feature_extractor:
    feature.append(item.feature)
    print(feature)


[Response(response='NO', feature='vanes')]
['vanes']


In [ ]:
loader = DirectoryLoader("RAG")
raw_docs = loader.load()
textsplitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = textsplitter.split_documents(raw_docs)
db = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = db.as_retriever()
 

prompt_for_rag = ChatPromptTemplate.from_template("""You are an assistant that is an expert at RAG (Retrieval Augmented Generation). You have been tasked with looking for the definition of the word provided to you as input and generate a summary of the available information."
            "You will use only the context provided to you.                                 
            Context:{context}
            Response:{Response}""")
rag_chain = ({"context": retriever, "Response": RunnablePassthrough()}) | prompt_for_rag | chat | StrOutputParser()

In [ ]:
#Base prompt chain, generates the first yaml version with minimal context
base_prompt = ChatPromptTemplate.from_messages([("system", system1), MessagesPlaceholder(variable_name="messages")])
yaml_generator = base_prompt | chat 
rag_output = rag_chain.invoke(f"What is the definition of {feature}?")

def chain_yes():
    chain = yaml_generator | StrOutputParser()
    yaml = chain.invoke({"messages": [human]})
    print("RAG not required, generating YAML")
    print()
    print(yaml)
    print()
    return yaml

def chain_no():
    chain = yaml_generator | StrOutputParser()
    ai = AIMessage(content=rag_output)
    yaml = chain.invoke({"messages":[ai]})
    print("Getting more information using RAG, then generating YAML")
    print()
    print(yaml)
    print()
    return yaml

def decision_maker(response_obj: Response):
    if response_obj.response == "YES":
       return chain_yes()
    elif response_obj.response == "NO":
        return chain_no()

output = ""
for response in feature_extractor:
    answer = decision_maker(response)
    output += answer



In [ ]:


#This chain ise used for the reflection prompt, which asks the model to critique and improve on the generated YAML text.
reflection_prompt = ChatPromptTemplate.from_messages([("system", refsystem), few_shot_prompt, ("ai", rag_output), MessagesPlaceholder(variable_name="messages"), ])
reflect = reflection_prompt | chat 

async def generation_node(state: Sequence[BaseMessage]):
        return await yaml_generator.ainvoke({"messages": state}, {"tags": ["loop 004"]})

async def reflection_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
        # Other messages we need to adjust
        cls_map = {"ai": HumanMessage, "human": AIMessage}
        # First message is the original user request. We hold it the same for all nodes
        translated = [state[0]] + [cls_map[msg.type](content=msg.content) for msg in state[1:]]
        res = await reflect.ainvoke({"messages": translated})
        # We treat the output of this as human feedback for the generator
        return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("reflect")

def should_continue(state: List[BaseMessage]):
    if len(state) > 2:
        return END
    return "generate"

builder.add_conditional_edges("reflect", should_continue)
builder.add_edge("generate", "reflect")
graph = builder.compile()

async for event in graph.astream([HumanMessage(content=output)]):
        for key, value in event.items():
            if isinstance(value, dict):
                print(value.get('content'))
                print('---')
                print()
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        print(item.get('content'))
                        print('---')
                        print()
                    else:
                        print(item.content)
                        print('---')
                        print()
            
